In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats
from IPython.display import display, IFrame

set_matplotlib_formats("svg")
sns.set_context("poster")
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)
pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

# Lecture 5 – Exploring and Cleaning Data

## DSC 80, Fall 2023

## 📣 Announcements 📣

- Project 1 due Wed!
- Lab 3 out, due on Mon

## 📆 Agenda

- [ ] Introduce dataset
- [ ] Introduce `plotly`
- [ ] Statistical vs. computational data types
- [ ] Data cleaning
    - [ ] Data quality checks
    - [ ] Missing data
    - [ ] Transformations and timestamps
    - [ ] Modifying structure
- [ ] Investigating student-submitted questions

## San Diego Food Safety

> In the last three years, one third of San Diego County restaurants have had at least one major food safety violation.

https://inewsource.org/2023/02/09/san-diego-restaurants-food-safety-violations/

(https://archive.ph/gz8BL)

### 99% Of San Diego Restaurants Earn ‘A' Grades, Bringing Usefulness of System Into Question

> Food held at unsafe temperatures. Employees not washing their hands. Dirty countertops. Vermin in the kitchen. An expired restaurant permit.
> 
> Restaurant inspectors for San Diego County found these violations during a routine health inspection of a diner in La Mesa in November 2016. Despite the violations, the restaurant was awarded a score of 90 out of 100, the lowest possible score to achieve an ‘A’ grade.

https://www.nbcsandiego.com/news/local/99-of-san-diego-restaurants-earn-a-grades-bringing-usefulness-of-system-into-question/25381/

(https://archive.ph/yB6RU)

### The Data

https://www.sandiegocounty.gov/content/sdc/deh/fhd/ffis/intro.html.html

- Had to download the data as JSON, then process into dataframes (will cover in future weeks!)
- Downloaded the 1000 restaurants closest to UCSD.


In [ ]:
rest = pd.read_csv('data/restaurants.csv')
insp = pd.read_csv('data/inspections.csv')
viol = pd.read_csv('data/violations.csv')

### Understanding the Data

![](imgs/ds-lifecycle.svg)

### Aside: Working with files

- So far, all data came in CSV files that loaded without problem.
- But many different formats and possible issues in loading in data!
- See [Chapter 8 of Learning DS](https://learningds.org/ch/08/files_intro.html) for more.

### You Try: Looking at the Data

- The articles said that one third of restaurants had at least one major safety violation.
- Which dataframes and columns seem most useful to verify this?

In [ ]:
# Fill me in

## Using `plotly` for Data Visualization

I've used `plotly` before in class, but let's talk about it now.

- Library for interactive data visualizations
- Install with `conda install plotly`
    - Discussion this week: why use `conda install` instead of `pip install`?

### `plotly.express` Syntax

`plotly` is very flexible but can be verbose. We use `plotly.express` to make plots quickly.

- Docs: https://plotly.com/python/plotly-express


In [ ]:
# Will include this at the top of each notebook from now on.
import plotly.express as px

# DSC 80 preferred styles, but not necessary
import plotly.graph_objects as go
import plotly.io as pio
pio.templates["dsc80"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        width=600,
        height=400,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+dsc80"

In [ ]:
fig = px.histogram(insp['score'])
fig

In [ ]:
scores = (
    insp[['grade', 'score']]
    .dropna()
    .groupby('grade')
    .mean()
    .reset_index()
)
px.bar(scores, x='grade', y='score')

## Feature Types

### Different Kinds of data

<center><img src='imgs/data-types.png' width=90%></center>

### Discussion Question

Determine the kind of each of the following variables:

- `insp['score']`
- `insp['grade']`
- `viol['violation_accela']`
- `viol['major_violation']`
- `rest['business_id']`
- `rest['opened_date']`

### Feature types vs. data types

- The data type `pandas` uses is not the same as the "data type" we talked about just now!
    - Difference between feature type vs. computational data type.
- Take care when the two don't match up very well!

In [ ]:
# pandas stores as int, but actually nominal
rest['business_id']

In [ ]:
# pandas stores as string, but actually numeric
rest['opened_date']

## Data Cleaning: Quality Checks

We often start an analysis by checking the quality of the data.

- Scope: Do the data match your understanding of the population? 
- Measurements and values: Are the values reasonable?
- Relationships: Are related features in agreement?
- Analysis: Which features might be useful in a future analysis? 

### Scope

In [ ]:
rest.head(2)

### Measurements and Values

In [ ]:
insp['grade'].value_counts()

In [ ]:
insp.info()

In [ ]:
(rest[rest.duplicated(subset=['address'])]
 .sort_values('address')
)

### Relationships

In [ ]:
# Useful to check whether ZIP codes match!
rest[['address', 'zip']]

In [ ]:
insp[['score', 'grade']]

### Analysis

- Most interested in:
    - `rest` df: `business_id`, `name`, `address`, `opened_date` from `rest`
    - `insp` df: `business_id`, `inspection_id`, `score`, `grade`, `completed_date`, `status`
    - `viol` df: `inspection_id`, `violation`, `major_violation`, `violation_text`, `violation_accela`
- Also, let's rename a few columns to make them easier to work with.

### 💡 Pro-Tip: Using `pipe`

When we manipulate dataframes, we prefer to define individual functions for each step, then use `df.pipe` to chain them all together.

- In practice, we would add functions one by one to the top of a notebook, then `pipe` them all.
- For today, will keep re-running `pipe` to show data cleaning process.

In [ ]:
def subset_rest(rest):
    return rest[['business_id', 'name', 'address', 'opened_date']]

rest = (
    pd.read_csv('data/restaurants.csv')
    .pipe(subset_rest)
)

In [ ]:
def subset_insp(insp):
    return (
        insp[['business_id', 'inspection_id', 'score', 'grade', 'completed_date', 'status']]
        .rename(columns={'completed_date': 'date'})
    )

insp = (
    pd.read_csv('data/inspections.csv')
    .pipe(subset_insp)
)

In [ ]:
def subset_viol(viol):
    return (
        viol[['inspection_id', 'violation', 'major_violation', 'violation_accela']]
        .rename(columns={'violation': 'kind',
                         'major_violation': 'is_major',
                         'violation_accela': 'violation'})
    )

viol = (
    pd.read_csv('data/violations.csv')
    .pipe(subset_viol)
)

### Joining the Restaurant Data

Let's join all three dataframes together so that we have all the data in one dataframe:

In [ ]:
def merge_all_restaurant_data():
    return (
        rest
        .merge(insp, on='business_id', how='left')
        .merge(viol, on='inspection_id', how='left')
    )

df = merge_all_restaurant_data()
df

## Data Cleaning: Looking for Missing Values

Next, it's important to check for missing values, as they can have a big effect on your analysis.

In [ ]:
insp[['score', 'grade']]

In [ ]:
df[df['inspection_id'].isna()]

In [ ]:
df.query('score == 0')

Many ways of handling missing values which we'll go into soon, but you must check to see how many you have!

## Data Cleaning: Transformations and Timestamps

It's often useful to look at ways of transforming your data to make it easier to work with, for example:

- Type conversions (changing the string `"$2.99"` to the number `2.99`).
- Converting units (feet to meters)
- Extraction (Getting `'vermin'` out of `'Vermin Violation Recorded on 10/10/2023'`).

Most commonly, we'll parse dates into `pd.Timestamp` objects.

In [ ]:
# This magical string tells Python what format the date is in.
# For more info: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
date_format = '%Y-%m-%d'

pd.to_datetime(insp['date'], format=date_format)

In [ ]:
# Another advantage of defining functions is that we can reuse this function
# for the opened_date column in `rest` if we wanted to
def parse_dates(insp, col):
    date_format = '%Y-%m-%d'
    dates = pd.to_datetime(insp[col], format=date_format)
    return insp.assign(**{col: dates})

insp = (
    pd.read_csv('data/inspections.csv')
    .pipe(subset_insp)
    .pipe(parse_dates, 'date')
)

# Should also remake df, since it depends on insp
df = merge_all_restaurant_data()

In [ ]:
insp['date']

### Working with Timestamps

- We often want to adjust granularity of timestamps to see overall trends, or seasonality.
- Use the `resample` method in `pandas`.
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In [ ]:
(insp.resample('2W', on='date')
 .size()
 .plot()
);

### The `.dt` accessor

- Like with Series of strings, `pandas` has a `.dt` accessor for properties of Timestamps.
- https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors

In [ ]:
insp['date'].dt.day

In [ ]:
insp['date'].dt.dayofweek

In [ ]:
dow_counts = (
    insp.assign(dow=insp['date'].dt.dayofweek)
    .groupby('dow')
    .size()
)
px.bar(dow_counts)

## Data Cleaning: Modifying Structure

We often reshape the dataframe's structure to make it more convenient for analysis. For example, we can:

- Simplify structure by removing columns or taking a set of rows for a particular period of time or geographic area.
    - Already did this!
- Adjust granularity by aggregating rows together.
- Reshape structure, most commonly by using `df.melt()` to un-pivot a dataframe.

### Using `melt`

- `melt()` is common enough that we'll give it a special mention.
- Often encounter pivot tables (esp from government data), which we call *wide* data, and we want to reshape into *long* form:

<center><img src='imgs/wide-vs-long.svg' width=40%></center>

- With long-form data, we can group by both year and month. Hard to do that with the wide-form data!
- Long-form data is also called *tidy* data.

## Student-Submitted Questions

- Come up with a question or two that you want to answer with the data.
    - E.g. "What are the most common violations around UCSD?"
- I will (attempt to) answer them live and explain as we go!
- Will go until no more questions or we run out of time.
- https://wall.sli.do/event/g2dESFa2co9kwmUwqfuCNL?section=e82e38eb-254e-4eb9-9de3-341347c52119

### Fill this in with student questions

...

### Example question: What are the most common violations?

In [ ]:
fig = px.bar(
    viol['kind'].value_counts().sort_values(),
    orientation='h'
)
fig.update_layout(width=1000, height=800)

## Takeaways

- Data cleaning is a necessary starting step in data analysis. 4 main things to watch out for:
    - Quality checks
    - Missing values
    - Transformations and timestamps
    - Modifying structure